# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
pip install catboost


In [2]:
pip install lightgbm

In [3]:
import pandas as pd
import warnings
import numpy as np
warnings.simplefilter('ignore')
from scipy import stats as st
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg' 
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from sklearn.utils import shuffle
from sklearn.model_selection import (
    GridSearchCV, 
    RandomizedSearchCV,
    train_test_split
)
import lightgbm
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (StandardScaler)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
np.random.seed(0)

import catboost
from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostRegressor, cv
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler
)

In [4]:
try :
    data  = pd.read_csv('/datasets/autos.csv',parse_dates=['DateCrawled','DateCreated','LastSeen'])
    
except:
    data  = pd.read_csv('autos.csv',parse_dates=['DateCrawled','DateCreated','LastSeen'])
    
display(data.head(10))


In [5]:
#функция для получения общей информации о датафрейме
def df_inf(data): 
    display(data.head())
    display(data.describe(include='all').T)
    display(data.info())
    for column in data.columns:
        display()
        display(column)
        display(data[column].unique())
    display(data[data.duplicated()])
    display(pd.DataFrame(round(data.isna().mean()*100,1)).style.background_gradient('coolwarm') )

In [6]:
df_inf(data)

Видим 1.нужно преобразовать время(преобразовали)

2.нужно преобразовать столбцы с int64 - где то можно сделать int9, где то int32 

3 обнаружены неправдоподобные данные в RegistrationYear,
'Power', возм 'Model' дубликаты

4 много пропусков

5 'NumberOfPictures' всегда 0

6 почтовый код явно не нужен 

In [7]:
data[['Price','PostalCode']].corr()

Видим очень слабую корреляцию цены с почтовым кодом , удалим его.

In [8]:
data.query('NumberOfPictures>0')

Видим что у всех строк число фотографий 0 , значит столбец не информативен удалим его

In [9]:
data=data.drop(['NumberOfPictures','PostalCode'],axis=1)
data.head()

In [10]:
data

In [11]:
data.astype({'Price': 'int32','RegistrationYear': 'int32','Power': 'int32','RegistrationMonth':'int8'}).dtypes

In [12]:
data.info()

преобразовали типы данных несколько снизив использование памяти

In [13]:
data['Repaired']=data['Repaired'].fillna('noinf')


In [14]:
data[data['VehicleType'].isna()].head(10)

In [15]:
data[data['FuelType'].isna()].head(10)

Видим что пропуски не имеют какой либо явной зависимости. Для того чтобы не потерять эти данные создадим в пропусках отдельную категорию noinf. Кроме того это поможет предсказать цену клиентам которые не указывают данную информацию

In [16]:
data.query('FuelType.isna()' )

In [17]:
data.query('FuelType.isna() and Model=="golf"' )

In [18]:
data.query(' Model=="golf"' )

In [19]:
data['FuelType'].unique()

In [20]:
data.groupby('Model')['Model'].value_counts().sort_values(ascending= False).tail(15)

In [21]:
data.query('Model=="materia"')

Видим что для строк где известна модель машины с более менее высокой точностью можно восстановить пропуски  Gearbox FuelType Power , также где Power=0 заменим медианой по модели машины

In [22]:
for model in data['Model'].unique():
    data.loc[(data['Power']==0)&(data['Model']==model),'Power'] = data.loc[data['Model'] == model]['Power'].median()

In [23]:
for model in data['Model'].unique():
    data.loc[(data['Gearbox'].isna())&(data['Model']==model),'Gearbox'] = data.loc[data['Model'] == model]['Gearbox'].mode()

In [24]:
for model in data.query('FuelType.isna()')['Model'].unique():
    data.loc[(data['FuelType'].isna())&(data['Model']==model),'FuelType'] = data.loc[data['Model'] == model]['FuelType'].mode()

In [25]:
for model in data['Model'].unique():
    data.loc[(data['Power'].isna())&(data['Model']==model),'Power'] = data.loc[data['Model'] == model]['Power'].median()

оставшиеся пропуски если есть заменим 

In [26]:
for m in [ 'VehicleType','Model','Repaired']:
    
    data[m]=data[m].fillna('noinf')
data['Gearbox']=data['Gearbox'].fillna('manual') 

In [27]:
display(pd.DataFrame(round(data.isna().mean()*100,1)).style.background_gradient('coolwarm') )

In [28]:
d=data['Model'].unique().tolist()
d=sorted(d)
print(d)

неявных дубликатов в моделях не обнаружено

In [29]:
data.query('Price==0')

Цена целевой признак поэтому обьявления с ценой 0 придется отбросить

In [30]:
data= data.query('Price!=0')

In [31]:
data['Price'].hist(bins=10, figsize=(12,9))

In [32]:
 data.query('Price<100')

In [33]:
 data= data.query('Price>100')

In [34]:
data['RegistrationYear'].unique()

In [35]:
data['RegistrationYear'].hist(bins=100, figsize=(12,9));

In [36]:
data=data.query('1970<=RegistrationYear<=2016')

К сожалению придется отбросить обьявления с неправдоподобным годом 

In [37]:
data['Power'].unique()

In [38]:
data.query("VehicleType in ['wagon'] and RegistrationYear==2003 and Model in ['focus']").head(10)

In [39]:
data.query("Power>500 ")

In [40]:
data.query("VehicleType in ['wagon'] and RegistrationYear==1996 and Model in ['a6']").head(10)

Видим сточки с неправдоподобным значением, заполним их медианой по марке машины

In [41]:
for model in data['Model'].unique():
    data.loc[(data['Power']>500 )&(data['Model']==model),'Power'] = data.loc[data['Model'] == model]['Power'].median()
    data.loc[(data['Power']==0 )&(data['Model']==model),'Power'] = data.loc[data['Model'] == model]['Power'].median()

In [42]:
data.loc[[1816,4060]]

In [43]:
data.query('Power== 0')

Заменили данные о мощности машины медианой по марке машины, в том числе машины где марка была неизвестна и была заменена на 'noinf'

In [44]:
display(pd.DataFrame(round(data.isna().mean()*100,1)).style.background_gradient('coolwarm') )

In [45]:
data['FuelType']=data['FuelType'].fillna('noinf')

Вывод  по шагу 1.

При изучении данных было выявлено множество пропусков и аномалий, также во многих столбцах можно было использовать менее затратные типы данных. Дубликаты не были выявлены. 
также были неправильно распознаны даты. 
Таким образом было преобразовано:
    время 
    столбцы с int64 - где то было сделано int8, где то int32
    были удалены неправдоподобные года регистрации в RegistrationYear
    были оставлены неправдоподобные данные в 'Power' так как это положитенльо влияло на корреляцию с ценой
    были заменены пропуски на noinf
    удалены лишние столбцы 'NumberOfPictures' так как там  всегда 0 и PostalCode так как  почтовый код явно не нужен 


## Обучение моделей

Выберем следующие регрессоры для обучения модели:

LinearRegression


RandomForestregressor
LGBMRegressor
Выберем следующие регрессоры для обучения модели:


В качестве метрики для всех моделей будем использовать MSE и в финальной таблице переведем в RMSE. 

In [46]:
data.head()

In [47]:
data=data.drop(['DateCrawled','DateCreated','LastSeen','RegistrationMonth','RegistrationYear'],axis=1)

In [48]:
data.head()

Было решено что часть признаков лишние и не будут способствовать повышению качества моделей. Эти признаки были отброшены

для моделей которые требуют кодирования признаков признаки назвали target features,
для остальных моделей признаки назвали target_cat features_cat

In [49]:

target = data['Price']
features = data.drop('Price', axis=1)

In [50]:
target_cat=data['Price']
features_cat=data.drop('Price', axis=1)

In [51]:
features_train_cat, features_valid_cat, target_train_cat, target_valid_cat = train_test_split(
features_cat, target_cat, test_size=0.4, random_state=12345)
features_valid_cat,features_test_cat,target_valid_cat, target_test_cat=train_test_split(features_valid_cat,target_valid_cat,test_size=0.50,random_state=12345)
print('Размер тренерующей выборки', features_train_cat.shape[0])
print('Размер валидационной выборки', features_valid_cat.shape[0])
print('Размер тестовой выборки', features_test_cat.shape[0])

In [52]:
features_train_ohe, features_valid_ohe, target_train_ohe, target_valid_ohe = train_test_split(
features, target, test_size=0.4,random_state=12345)
features_valid_ohe,features_test_ohe,target_valid_ohe, target_test_ohe=train_test_split(features_valid_ohe,target_valid_ohe,test_size=0.50,random_state=12345)
print('Размер тренерующей выборки', features_train_ohe.shape[0])
print('Размер валидационной выборки', features_valid_ohe.shape[0])
print('Размер тестовой выборки', features_test_ohe.shape[0])

In [53]:
#категориальные признаки для OHE 
ohe_features = features_train_ohe.select_dtypes(include='object').columns.to_list()
print(ohe_features)
num_features=['Price','Power','Kilometer']
print(num_features)

In [54]:
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
# обучаем энкодер на заданных категориальных признаках тренировочной выборки
encoder_ohe.fit(features_train_ohe[ohe_features])
# добавляем закодированные признаки в X_train_ohe
# encoder_ohe.get_feature_names_out() позволяет получить названия колонок
features_train_ohe[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_train_ohe[ohe_features])

# удаляем незакодированные категориальные признаки (изначальные колонки)
features_train_ohe = features_train_ohe.drop(ohe_features, axis=1)



In [55]:
# энкодером, который обучен на ТРЕНИРОВОЧНОЙ ВЫБОРКЕ, кодируем валидационную и тестовую
features_valid_ohe[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_valid_ohe[ohe_features])

features_valid_ohe = features_valid_ohe.drop(ohe_features, axis=1)

features_test_ohe[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_test_ohe[ohe_features])

features_test_ohe = features_test_ohe.drop(ohe_features, axis=1)



In [56]:
features_valid_ohe.head()

In [57]:
model_cat=CatBoostRegressor(cat_features=[ 'VehicleType', 'Gearbox',
                'Model', 'FuelType', 'Brand','Repaired'])
model_rfr =  make_pipeline(RandomForestRegressor())
model_lr = make_pipeline(LinearRegression())
model_LGBM= LGBMRegressor() 
n_iter_search = 12 

In [58]:
parameters_cat = {'learning_rate':[0.5,0.6],
                  'iterations':[100,200],
                  'random_state':[12345],
                  'depth': [4, 6]
                 }
parameters_lr = {'linearregression__normalize':[False]}
parameters_rfr = {'randomforestregressor__n_estimators':[5,10],
                  'randomforestregressor__max_depth':[3,8],
                  'randomforestregressor__random_state':[12345]}
parameters_LGBM={'num_leaves':[31, 100, 200], 
                'learning_rate':[0.1, 0.3, 0.5],
                'random_state':[12345]}


Проверим как маштабирование признаков влияет на линейные модели

In [59]:
%%time
regressor = LinearRegression()


cv_RMSE_LR = (cross_val_score(regressor, 
                             features_train_ohe, 
                             target_train_ohe, 
                             cv=5, 
                             scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('Mean RMSE from CV of LinearRegression =', cv_RMSE_LR)

In [60]:
%%time

regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])

cv_RMSE_LR_S = (cross_val_score(pipeline,
                               features_train_ohe, 
                                target_train_ohe, 
                                cv=5, 
                                scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('Mean RMSE from CV of LinearRegression =', cv_RMSE_LR_S)

Видим что масштабирование признаков ухудшает модель. Откажемся от применение масштаба.

In [61]:
model=LinearRegression()
random_search = RandomizedSearchCV(model_lr, 
                                   param_distributions=parameters_lr, 
                                   n_iter=10, 
                                   cv=3, 
                                   scoring = 'neg_mean_squared_error',
                                      ) 
random_search.fit(features_train_ohe, target_train_ohe) 
lr_final =random_search.best_score_
print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка : {}".format(random_search.best_score_))


random_search.cv_results_

In [62]:

random_search = RandomizedSearchCV(model_cat, 
                                   param_distributions=parameters_cat, 
                                   n_iter=10, 
                                   cv=2, 
                                   verbose=True,scoring = 'neg_mean_squared_error'
                                   
                                  )
                                   
                                     
random_search.fit(features_train_cat, target_train_cat) 
Cat_final =random_search.best_score_
print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка : {}".format(random_search.best_score_))


In [63]:
%%time
random_search = RandomizedSearchCV(model_LGBM, 
                                   param_distributions=parameters_LGBM, 
                                   n_iter=10, 
                                   cv=3, 
                                   scoring = 'neg_mean_squared_error',
                                   
                                      ) 
random_search.fit(features_train_cat.astype('category'), target_train_cat) 
LGBM_final =random_search.best_score_
print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка : {}".format(random_search.best_score_))

%%time
categorical_feats=[ 'VehicleType', 'Gearbox',
                'Model', 'FuelType', 'Brand','Repaired']
data_1=data.copy()

target_LGBMR=data_1['Price']
features_LGBMR=data_1.drop('Price', axis=1)
features_train_LGBMR, features_valid_LGBMR, target_train_LGBMR, target_valid_LGBMR = train_test_split(
    features_LGBMR, target_LGBMR, test_size=0.25, random_state=12345)

model_LGBM_1= LGBMRegressor(categorical_feats=[ 'VehicleType', 'Gearbox',
                'Model', 'FuelType', 'Brand','Repaired']) 
random_search = RandomizedSearchCV(model_LGBM_1, 
                                   param_distributions=parameters_LGBM, 
                                   n_iter=10, 
                                   cv=3, 
                                   scoring = 'neg_mean_squared_error',
                                   
                                      ) 
random_search.fit(features_train_LGBMR.astype('category'), target_train_LGBMR) 
LGBM_final =random_search.best_score_
print("Лучшие параметры: {}".format(random_search.best_params_)) 
print("Лучшая оценка : {}".format(random_search.best_score_))

%%time
random_search_rfr_rougher = RandomizedSearchCV(model_rfr, param_distributions=parameters_rfr , n_iter=n_iter_search, cv=5, n_jobs=-1,scoring = 'neg_mean_squared_error')
random_search_rfr_rougher.fit(features_train_cat, target_train_cat)
rfr_rougher =random_search_rfr_rougher.best_score_
# Вывод результатов
print("Лучшие параметры: {}".format(random_search_rfr_rougher.best_params_)) 
print("Лучшая оценка : {}".format(random_search_rfr_rougher.best_score_))


Вывод по шагу 2
Данные были избавлены от неважных признаков, были разбиты на обучающие и валидационные выборки
Были изучены 4  модели. Зафиксированы лучшие гиперпараметры и время их выполнения.

## Анализ моделей

In [64]:
%%time
model = LinearRegression(normalize=False)
model.fit(features_train_ohe, target_train_ohe)

In [65]:
%%time
target_predict = model.predict(features_valid_ohe)

In [66]:
final_RMSE_LR = mean_squared_error(target_valid_ohe, target_predict) ** 0.5

In [67]:
%%time

model = CatBoostRegressor(learning_rate=0.6, 
                          random_state=12345, 
                          iterations=200,
                         depth=6,
                          cat_features=[ 'VehicleType', 'Gearbox',
                'Model', 'FuelType', 'Brand','Repaired']) 
model.fit(features_train_cat, target_train_cat)

In [68]:
%%time
target_predict = model.predict(features_valid_cat)

In [69]:
final_RMSE_CBR_ordinal = mean_squared_error(target_valid_cat, target_predict) ** 0.5

In [85]:
%%time
model = LGBMRegressor(learning_rate=0.1, 
                      num_leaves=200, 
                      random_state=12345)
model.fit(features_train_ohe, target_train_ohe)

In [84]:
mp = pd.Series(model.feature_importances_,
                         features_train_cat.columns)

fig, ax = plt.subplots(figsize=(10,10))
mp.plot.bar(ax=ax)
ax.set_title("Важность признаков")
ax.set_ylabel('Важность')
fig.tight_layout()

In [86]:
%%time
target_predict = model.predict(features_valid_ohe)

In [87]:
final_RMSE_LGBMR = mean_squared_error(target_valid_ohe, target_predict) ** 0.5

In [74]:
%%time
model = RandomForestRegressor(random_state=12345,
                             n_estimators=10,
                             max_depth=8)
model.fit(features_train_ohe, target_train_ohe)

In [75]:
%%time
target_predict = model.predict(features_valid_ohe)


final_RMSE_RFR = mean_squared_error(target_valid_ohe, target_predict) ** 0.5

Сравним результаты моделей на валидационной выборке

In [88]:
index = ['LinearRegression',
         
         
         'CatBoostRegressor ',
         
         'LGBMRegressor'
        ]
d = {'RMSE модели на валидационной выборке':[final_RMSE_LR,
                                           final_RMSE_CBR_ordinal,
                                                
                                              
                                                final_RMSE_LGBMR
                                           
                                               ],
        'Время обучения , сек':[2.81,
                                     16.3 ,
                                     1.53
                                     
                                     ],
        'Время предсказания модели, сек':[0.081,
                                          0.18,
                                          0.373
                                          
                                         ]
       }
scores_data = pd.DataFrame(data=d, index=index)

In [89]:
display(scores_data)

In [90]:
%%time
model = LGBMRegressor(learning_rate=0.1, 
                      num_leaves=200, 
                      random_state=12345)
model.fit(features_train_cat.astype('category'), target_train_cat)

In [95]:
%%time
target_predict = model.predict(features_test_cat.astype('category'))

In [96]:
final_RMSE_LGBMR = mean_squared_error(target_test_ohe, target_predict) ** 0.5

In [97]:
print( 'итоговый результат модели на тестовой выборке' ,final_RMSE_LGBMR)

Итоговый вывод
В ходе работы было выполнено:

Загружены данные и проведена предобработка.
Выполнено сравнение моделей с использованием различных наборов гиперпараметров.
Все модели были проверены на валидационной выборке 

Лучшей моделью  показал себя LGMRegressor. Следом за ним идет CatBoostRegressor . Остальные модели не набрали необходимую точность. По скорости обучения и предсказания LGMRegressor также бесусловный лидер он обучался быстрее всех моделей даже линейной  и почти в 10 раз быстрее CatBoostRegressor

Таким образом выбирается модель LGMRegressor
Также посчитали ее RMSE на итоговой выборке он составил 2063

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей